In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
from sklearn import metrics 
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/farec/train.csv")

df = df.replace("correct", 1).replace("incorrect", 0)
df['pickup_time'] = pd.to_datetime(df['pickup_time'])
df['pickup_year'] = pd.DatetimeIndex(df['pickup_time']).year
df['pickup_month'] = pd.DatetimeIndex(df['pickup_time']).month
df['pickup_day'] = pd.DatetimeIndex(df['pickup_time']).day


df = df.drop("pickup_time", axis=1).drop("drop_time", axis=1).drop("tripid", axis=1)

df=df.fillna(df.mean())
print(df.head())

In [ ]:
train, test = train_test_split(df, test_size = 0.25, random_state=200)

train_x = train.drop("label", axis=1)
train_y = train.label
test_x = test.drop("label", axis=1)
test_y = test.label

gnb = GaussianNB()
gnb.fit(train_x, train_y)
predict_y_gnb = gnb.predict(test_x)

accuracy = metrics.accuracy_score(test_y, predict_y_gnb)

print('gnb accuracy- ', accuracy)

In [ ]:
df2 = pd.read_csv("/kaggle/input/farec/test.csv")

tripid = df2.tripid
df2['pickup_time'] = pd.to_datetime(df2['pickup_time'])
df2['pickup_year'] = pd.DatetimeIndex(df2['pickup_time']).year
df2['pickup_month'] = pd.DatetimeIndex(df2['pickup_time']).month
df2['pickup_day'] = pd.DatetimeIndex(df2['pickup_time']).day

df2 = df2.drop("pickup_time", axis=1).drop("drop_time", axis=1).drop("tripid", axis=1)
predictions = gnb.predict(df2)


output = {'tripid' : tripid, 'prediction' : predictions}
result = pd.DataFrame(output)
print(result.head())

result.to_csv('submission.csv', index= False)